In [180]:
import numpy as np
import random
import math

In [217]:
class HMM:
    def __init__(self,n,seed,o):
        self.N = n
        self.M = 27
        self.T = len(o)
        self.A = np.zeros((self.N,self.N))
        self.B = np.zeros((self.N,self.M))
        self.pi = np.zeros((1,self.N))
        self.initialize(seed)
        self.minIters = 100
        self.oldlogProb = -1.0
        # New LogProb 
        self.logProb = 0.0
        self.α = np.zeros((self.T,self.N),dtype=np.float64)
        self.β = np.zeros((self.T,self.N),dtype=np.float64)
        self.ϒ = np.zeros((self.T,self.N))
        self.diGamma = np.zeros((self.T,self.N,self.N))
        self.c = np.zeros((1,self.T))
        
# Initialize A, B and pi    
    def initialize(self,seed):
        random.seed(seed)        
        # Initialize A        
        prob = 1.0 / self.N;
        for i in range(0,self.N):
            sumN = 0.0;
            for j in range(0,self.N):
                if(random.randint(1,100)%2 == 0):
                    self.A[i][j] = prob + float(random.randint(1,3999)*prob)/100000
                else:
                    self.A[i][j] = prob - float(random.randint(1,3999)*prob)/100000
                sumN = sumN + self.A[i][j];
            for j in range(0,self.N):
                self.A[i][j] = self.A[i][j]/sumN;
        print("Initial A =",self.A)
        
        # Initialize B       
        prob = 1.0 / self.M;
        for i in range(0,self.N):
            sumM = 0.0;
            for j in range(0,self.M):
                if(random.randint(1,100)%2 == 0):
                    self.B[i][j] = prob + float(random.randint(1,3999)*prob)/100000
                else:
                    self.B[i][j] = prob - float(random.randint(1,3999)*prob)/100000  
                sumM = sumM + self.B[i][j];
            for j in range(0,self.M):
                self.B[i][j] = self.B[i][j]/sumM;
        
        # Initialize pi       
        prob = 1.0 / self.N;
        sumN = 0.0;
        for i in range(0,self.N):    
            if(random.randint(1,100)%2 == 0):
                self.pi[0][i] = prob + float(random.randint(1,3999)*prob)/100000
            else:
                self.pi[0][i] = prob - float(random.randint(1,3999)*prob)/100000
            sumN = sumN + self.pi[0][i];
        for i in range(0,self.N):
            self.pi[0][i] = self.pi[0][i]/sumN;
        print("Initial B=",self.B)
        print("Initial Pi=",self.pi)
    
    def alphaPass(self,o):
        # Compute α0(i)
        self.c[0][0] = 0
        for i in range(0,self.N):
            self.α[0][i] = self.pi[0][i]*self.B[i][o[0]]
            self.c[0][0] = self.c[0][0] + self.α[0][i]
        
        # Scale the α0(i)
        self.c[0][0] = 1/self.c[0][0]
        for i in range(0,self.N):
             self.α[0][i] = self.c[0][0]* self.α[0][i]
        
        # Compute αt(i)
        for t in range(1,self.T):
            self.c[0][t] = 0
            for i in range(0,self.N):
                self.α[t][i] = 0
                for j in range(0,self.N):
                    self.α[t][i]  = self.α[t][i] + self.α[t-1][j]*self.A[j][i]
                self.α[t][i] = self.α[t][i]*self.B[i][o[t]]
                self.c[0][t] = self.c[0][t] + self.α[t][i]
                
            #  Scale αt(i)
            self.c[0][t]= 1/self.c[0][t]
            for i in range(0,self.N):
                self.α[t][i] = self.c[0][t]* self.α[t][i]
            
    def betaPass(self,o):
        for i in range(0,self.N):
            self.β[self.T-1][i] = self.c[0][self.T-1]
        
        # β-pass
        for t in range(self.T-2, -1, -1):
            for i in range(0,self.N):
                self.β[t][i] = 0
                for j in range(0,self.N):
                    self.β[t][i]  = self.β[t][i] + self.A[i][j]*self.B[j][o[t+1]]*self.β[t+1][j]
                
                # Scale β[t][i] with same scale factor as α[t][i]
                self.β[t][i] = self.c[0][t]*self.β[t][i]
    
    def gamma(self,o):
        for t in range(0,self.T-1):
            denom = 0
            for i in range(0,self.N):
                for j in range(0,self.N):
                    denom = denom + self.α[t][i]*self.A[i][j]*self.B[j][o[t+1]]*self.β[t+1][j]
            for i in range(0,self.N):
                self.ϒ[t][i] = 0
                for j in range(0,self.N):
                    self.diGamma[t][i][j] = (self.α[t][i]*self.A[i][j]*self.B[j][o[t+1]]*self.β[t+1][j])/denom
                    self.ϒ[t][i] = self.ϒ[t][i] + self.diGamma[t][i][j]
        
        # Special case for ϒ[T-1][i]
        denom = 0
        for i in range(0,self.N):
            denom = denom + self.α[self.T-1][i]
        for i in range(0,self.N):
            self.ϒ[self.T-1][i] = self.α[self.T-1][i]/denom
    
    def reEstimate(self,o):
        # Re-estimate pi
        for i in range(0,self.N):
            self.pi[0][i] = self.ϒ[0][i]

        # Re-estimate A
        for i in range(0,self.N):
            for j in range(0,self.N):
                numer = 0
                denom = 0
                for t in range(0,self.T-1):
                    numer = numer + self.diGamma[t][i][j]
                    denom = denom + self.ϒ[t][i]
                self.A[i][j] = numer/denom

        # Re-estimate B
        for i in range(0,self.N):
            for j in range(0,self.M):
                numer = 0
                denom = 0
                for t in range(0,self.T):
                    if(o[t]==j):
                        numer = numer + self.ϒ[t][i]
                    denom = denom + self.ϒ[t][i]
                self.B[i][j] = numer/denom

    def computeLog(self):
        logProb = 0.0
        for i in range(0,self.T):
            logProb = logProb + math.log(self.c[0][i])
        self.logProb = -logProb
        
    def control(self,o):
        iters = 0
        while(iters<self.minIters and (self.logProb > self.oldlogProb)):
            self.oldlogProb = self.logProb;
            # run once for first iter
            self.alphaPass(o)
            self.betaPass(o)
            self.gamma(o)
            self.reEstimate(o)
            self.computeLog()
            if(iters == 0):
                self.oldlogProb = self.logProb - 1.0
            iters = iters + 1
            print("Iteration Completed= {0}, log [P(O|λ)] ={1}".format(iters, self.logProb))
        print("Total iterations =",iters)
        print("log [P(O|λ)] =", self.logProb)
        print("Final pi =",self.pi)
        print("Final A =",self.A)
        print("Final B =",self.B)

In [218]:
def preProcess():
    text = ""
    with open('Book Corpus.txt', 'r') as myfile:
        text=myfile.read().replace('\n', '')
    newTextO = list()
    i = 0
    for char in text:
        if(ord(char.lower())>=97 and ord(char.lower())<=122):
            newTextO.append(ord(char.lower())%97)
            i = i+1
        else:
            if(ord(char.lower())==32):
                newTextO.append(26)
                i = i+1
        if(i==50000):
            break
    return newTextO

In [213]:
obSeq = preProcess()

In [ ]:
hmmModel = HMM(26,2,obSeq)
hmmModel.control(obSeq)

Initial A = [[0.03858201 0.037869   0.03959638 0.03978203 0.03883378 0.03877189
  0.03849399 0.0373647  0.03775906 0.0390571  0.03708295 0.03993232
  0.03929426 0.03764683 0.03701837 0.03706604 0.03901097 0.03990464
  0.037008   0.03777021 0.03869655 0.03871693 0.03807425 0.03871577
  0.03871116 0.03924083]
 [0.03860132 0.03909167 0.03832401 0.03928225 0.03918525 0.03945343
  0.03860817 0.03859219 0.03670196 0.03829205 0.0374175  0.03919171
  0.03906123 0.03843014 0.03760542 0.0388212  0.03933627 0.03907189
  0.03663843 0.03858725 0.03690928 0.03916851 0.03728207 0.03769482
  0.03931002 0.03934197]
 [0.03965591 0.03786052 0.03685181 0.03876664 0.03677984 0.03697776
  0.03739962 0.03748537 0.03924707 0.038919   0.03942737 0.03751446
  0.03885622 0.03925855 0.03951121 0.03941972 0.03970453 0.03711327
  0.03837349 0.03930526 0.03785287 0.03863687 0.03965438 0.03946412
  0.03806686 0.03789727]
 [0.03690536 0.03905438 0.03958339 0.03829965 0.03782248 0.03799676
  0.03835186 0.0382079  0.038

Iteration Completed= 1, log [P(O|λ)] =-164834.95274722096
Iteration Completed= 2, log [P(O|λ)] =-141596.16525311454
